In [1]:
import numpy as np
import sklearn 

import onlinecp.algos as algos
import onlinecp.utils.feature_functions as feat
import onlinecp.utils.gendata as gd
import onlinecp.utils.evaluation as ev
from onlinecp.utils.datasets import pw_linear, pw_constant

In [2]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
gd.stream_GMM(d=10, k=3, n=5000, nb_change=0)

In [ ]:
Y, _, _ = pw_linear(500, 4)

In [ ]:
Y.shape

In [ ]:
d=4
n=10000
X, cps, ground_truth = pw_constant(n, d, 4, noise_std=2)

In [ ]:
np.where(ground_truth==1)[0].min()

In [ ]:
X=X[:,:4]

In [ ]:
newma_stat

In [ ]:
newma_stat = np.array([i[0] for i in detector.stat_stored])

In [ ]:
online_th = np.array([i[1] for i in detector.stat_stored])

In [ ]:
np.where(ground_truth==1)

In [ ]:
plt.figure()
plt.scatter([i for i in range(newma_stat.shape[0])], newma_stat)
plt.scatter([i for i in range(newma_stat.shape[0])], online_th)
plt.xlabel('Time')
plt.ylabel('Stat')
plt.show()

In [ ]:
EDD, FA, ND = ev.compute_curves(ground_truth, detection_stat, num_points=30, start_coeff=1.05, end_coeff=1.2)
EDDth, FAth, NDth = ev.compute_curves(ground_truth, detection_stat, num_points=30,
                                      thres_values=online_th, start_coeff=1, end_coeff=1)

In [ ]:
ground_truth

In [ ]:
!  python test_algo_on_data.py 'newmaRFF' 'tyler'  -n 200 -nb 50 -d 1 -show 

In [ ]:
t = np.load('tyler.npz')

In [ ]:
t.__dict__

In [ ]:
ev.compute_curves()

In [ ]:
t['ground_truth']

In [ ]:
t['online_th']

In [ ]:
t['detection_stat']

In [ ]:
t['EDDth']

In [ ]:
Y, cps = pw_normal(1000, 2)

In [ ]:
cps[-1]

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
# X, y = make_blobs(n_samples=1000, centers=2, n_features=2, random_state=0)

In [ ]:
plt.figure()
plt.scatter(X[:,0], X[:,1])
# plt.plot(FAth, NDth, 'o', markersize=20)
plt.xlabel('False Alarm')
plt.ylabel('Missed Detection')
plt.show()

In [ ]:
np.array(cps)

In [ ]:
plt.figure()
plt.plot(t['FA'], t['ND'], '-o')
# plt.plot(FAth, NDth, 'o', markersize=20)
plt.xlabel('False Alarm')
plt.ylabel('Missed Detection')
plt.show()

In [ ]:
parser.add_argument('-n', type=int, default=2000, help='number of samples for each distribution')
parser.add_argument('-nb', type=int, default=500, help='number of changes of distribution in the series')
parser.add_argument('-d', type=int, default=100, help='dimensionality of the samples in the time series')

In [ ]:
# default data settings
n=200 # distance between cps, the first CP is at n, then 2n, 3n etc, last one is at n*(nb) - n
nb_change=2
d=2

In [ ]:
# parameters of gmm
k = 3
std_mean = 10  # the bigger, the more change in means
wishart = 5  # the bigger, the less change in diagonal variances

X, ground_truth = gd.stream_GMM(d=d, n=n, nb_change=nb_change, std_mean=std_mean, 
                                concentration_wishart=wishart, k=k)

In [ ]:
ground_truth.shape

In [ ]:
import pandas as pd

In [ ]:
plt.figure()
plt.plot([i for i in range(Y.shape[0])], pd.DataFrame(Y)[0].values, '-o')
# plt.plot(FAth, NDth, 'o', markersize=20)
plt.xlabel('False Alarm')
plt.ylabel('Missed Detection')
plt.show()

In [ ]:
plt.figure()
plt.plot([i for i in range(X.shape[0])], pd.DataFrame(X)[1].values, '-o')
# plt.plot(FAth, NDth, 'o', markersize=20)
plt.xlabel('False Alarm')
plt.ylabel('Missed Detection')
plt.show()

In [ ]:
# each row is a data point, each col is a feature/dimension
X.shape

In [ ]:
X1, _, _ = pw_constant(5000, 15, 0, 0.5)

In [38]:
X2, truth = gd.stream_GMM(d=15, nb_change=10, n=1000)

In [ ]:
X3, _= gd.stream_GMM(d=15, nb_change=0, n=1000)

In [5]:
from experiments import Experiment

In [27]:
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, 10000)

In [39]:
exp = Experiment(X2, 'ScanB', [5,6,7,8])

In [40]:
exp.run_algo()

Start algo  ScanB ... (can be long !)


In [41]:
exp.plot_stat_time_series()

In [ ]:
exp.plot_stat_distribution()

In [ ]:
from onlinecp.utils.mmd import linear_mmd, quad_mmd

In [ ]:
linear_mmd(X2, X3)

In [ ]:
quad_mmd(X2, X3, biased=False)

In [ ]:
class KCUSUM:
    def __init__(self, threshold, slack):
        """
        https://arxiv.org/pdf/1903.01661.pdf
        >>> sum(3,4)
        4
        :param threshold: # of standard deviations to detect a CP
        :param slack: # of stds to allow for noise in each sample
        """
        self.threshold = threshold
        self.slack = slack
        self.kcusum = 0
        self.kcusum_vector = []

    def predict(self, reference, signal, gamma=1):
        for i, _ in enumerate(signal):
            update = 0
            if (i % 2) == 0 and i != 0:
                update = linear_mmd(signal[i-2:i], reference[i-2:i], gamma) - self.slack
            self.kcusum = max(0, self.kcusum + update)
            self.kcusum_vector.append(self.kcusum)
            if self.kcusum > self.threshold:
                return i

    def reset(self):
        self.kcusum = 0
        self.kcusum_vector = []


In [ ]:
X1.shape

In [ ]:
kc = KCUSUM(0.5, 0)

In [ ]:
kc.predict(X2, X3)

In [ ]:
kc.kcusum_vector

In [ ]:
plt.figure()
plt.plot([i for i in range(len(kc.kcusum_vector))], kc.kcusum_vector)
plt.show()

In [ ]:
plt.figure()
# plt.plot(FAth, NDth, 'o', markersize=20)
plt.hist(kc.kcusum_vector)
plt.show()